In [1]:
import networkx as nx
import numpy as np
import pandas as pd
from math import sqrt
import csv
import sys

In [2]:
from tqdm.notebook import tqdm

In [3]:
from pyBallMapper_Bokeh import graph_GUI, read_graph_from_list

In [4]:
from matplotlib.colors import ListedColormap
from matplotlib import cm

In [5]:
from bokeh.plotting import figure, show

In [6]:
# to deal with large csv
maxInt = sys.maxsize
decrement = True

while decrement:
    # decrease the maxInt value by factor 10
    # as long as the OverflowError occurs.
    decrement = False
    try:
        csv.field_size_limit(maxInt)
    except OverflowError:
        maxInt = int(maxInt/10)
        decrement = True

# Jones up to 13

In [7]:
# table with the coloring functions
coloring_df = pd.read_csv('output/jones_13/Jones_upto_13_colors.csv', sep=' ')
coloring_df.index = range(1, len(coloring_df)+1)

coloring_df['signature_mod4'] = coloring_df.signature % 4


#coloring_df['span'] = coloring_df['maximum_exponent'] - coloring_df['minimum_exponent']

print(coloring_df.shape)

(12966, 8)


In [8]:
coloring_df.head()

,number_of_crossings,table_number,is_alternating,signature,minimum_exponent,maximum_exponent,norm,signature_mod4
1,0,1,1,0,0,0,1.000000,0
2,3,1,1,2,1,4,1.732051,2
3,4,1,1,0,-2,2,2.236068,0
4,5,1,1,4,2,7,2.236068,0
5,5,2,1,2,1,6,3.000000,2


In [25]:
%%time
# Prepare Data

# adj lists path
GRAPH1_PATH = 'output/jones_13/20_edges'

# point covered by each node path
GRAPH1_POINTS_PATH = 'output/jones_13/20_points_covered_by_landmarks'


###########
# GRAPH 1 #
###########

#Here we adopt standard colour palette
my_red_palette = cm.get_cmap(name='jet') # multicolor
#my_red_palette = cm.get_cmap(name='Reds') # monochrome

# read graph
# ASSUME NODES ARE NUMBERED FROM 1 TO N
G1 = read_graph_from_list(GRAPH1_PATH, GRAPH1_POINTS_PATH,
                          coloring_df[['number_of_crossings', 'norm',
                                       'signature']],
                          add_points_covered=False
                          )

# create a GUI with input our BM graph, 
# a dataframe with coloring functions (one value per point in the pointcloud)
# and a color palette
# in this case we use the pointcloud as coloring function
print('creating GUI')
my_fancy_gui = graph_GUI(G1, my_red_palette, 
                         coloring_df[['number_of_crossings', 'norm',
                                       'signature',]].columns.to_list(),
                         figsize=(800, 600),
                         render_iterations=2000)

loading edgelist
loading points covered


0it [00:00, ?it/s]

computing coloring


  0%|          | 0/97 [00:00<?, ?it/s]

creating GUI
CPU times: user 2.72 s, sys: 58.3 ms, total: 2.77 s
Wall time: 3.66 s


In [26]:
my_fancy_gui.color_by_variable('signature', MIN_VALUE=-12, MAX_VALUE=+12)
                               


color by variable signature 
MIN_VALUE: -12.000, MAX_VALUE: 12.000


(-12, 12)

In [27]:
show(my_fancy_gui.plot)

### Add a legend

In [28]:
from bokeh.models import FixedTicker, LinearColorMapper, LogColorMapper, ColorBar, BasicTicker, LogTicker
from matplotlib.colors import to_hex


In [29]:
# # # continuous colorbar
# num_ticks = 100
# low = 0
# high = 1

# color_mapper = LinearColorMapper(palette=[to_hex(my_red_palette(color_id)) 
#                                           for color_id in np.linspace(0, 1, num_ticks)], 
#                                  low=low, high=high)


# color_bar = ColorBar(color_mapper=color_mapper, 
#                      major_label_text_font_size='14pt',
#                      label_standoff=12,
#                     )


# # color_mapper = LogColorMapper(palette=[to_hex(my_red_palette(color_id)) 
# #                                           for color_id in np.linspace(0, 1, num_ticks)], 
# #                                  low=low, high=high)

# # log_ticks = LogTicker(mantissas=[1,2,3,4,5], desired_num_ticks=10)

# # color_bar = ColorBar(color_mapper=color_mapper, 
# #                      major_label_text_font_size='14pt',
# #                      label_standoff=12,
# #                      ticker=log_ticks
# #                     )

In [30]:
# discrete colorbar
num_ticks = 13
low = -13
high = 13
color_mapper = LinearColorMapper(palette=[to_hex(my_red_palette(color_id)) 
                                          for color_id in np.linspace(0, 1, num_ticks)], 
                                 low=low, high=high)

ticks = [i for i in range(-12, 13, 2)]
color_ticks = FixedTicker(ticks=ticks)

color_bar = ColorBar(color_mapper=color_mapper, 
                     major_label_text_font_size='14pt',
                     label_standoff=12,
                     ticker=color_ticks,
                    )

In [31]:
my_fancy_gui.plot.add_layout(color_bar, 'right')

In [32]:
show(my_fancy_gui.plot)